In [1]:
import os
import geopandas as gpd
import pandas as pd
from pyproj import CRS

In [13]:
def get_real_reach_length(working_directory: str, river_name: str) -> pd.DataFrame:
    """
    Calculate the real reach lengths from a shapefile containing river reach geometries.

    Parameters:
        working_directory (str): Base path to the data directory.
        river_name (str): Name of the river used to locate the shapefile.

    Returns:
        pd.DataFrame: DataFrame with reach lengths in meters.
    """
    # Construct the path to the shapefile
    shapefile_path = os.path.join(
        working_directory, "HydroATLAS", "HydroRIVERS", "Extracted_Rivers", river_name, f"{river_name}_reaches.shp"
    )

    # Load shapefile using geopandas
    gdf = gpd.read_file(shapefile_path)

    # Print the Coordinate Reference System (CRS)
    print("Original CRS:", gdf.crs)

    # Convert to WGS84 (lat/lon) for UTM zone determination
    gdf_wgs84 = gdf.to_crs(epsg=4326)

    # Calculate UTM zone from centroid longitude
    centroid = gdf_wgs84.geometry.unary_union.centroid
    lon = centroid.x
    utm_zone = int((lon + 180) / 6) + 1
    epsg_code = 32600 + utm_zone if centroid.y >= 0 else 32700 + utm_zone

    print("Reprojecting to: EPSG:", epsg_code)

    # Reproject to appropriate UTM zone
    gdf = gdf.to_crs(epsg=epsg_code)

    # Compute length of each reach in meters and add as new column
    gdf["real_reach_length_m"] = gdf.geometry.length

    # Return DataFrame with geometry removed, showing lengths
    return gdf.drop(columns=["geometry", "reach_len", "ds_dist"])

In [14]:
working_directory = r"D:\Dissertation\Data"
river_name = 'Bermejo'
real_reach_lengths = get_real_reach_length(working_directory, river_name)

Original CRS: EPSG:3395
Reprojecting to: EPSG: 32720


In [16]:
real_reach_lengths.head()

,ds_order,real_reach_length_m
0,20,64178.626636
1,19,63825.781514
2,18,63475.973818
3,17,63302.532922
4,16,62962.332716


In [19]:
real_reach_lengths

,ds_order,real_reach_length_m
0,20,64178.626636
1,19,63825.781514
2,18,63475.973818
3,17,63302.532922
4,16,62962.332716
5,15,46977.787324
6,14,46823.985123
7,13,46389.296791
8,12,46212.078380
9,11,45389.961886


In [42]:
def calculate_xtran(
    working_directory: str,
    river_name: str,
    real_reach_lengths: pd.DataFrame,
    bulk_density: float
) -> pd.DataFrame:
    """
    Calculate cross-channel translation distances (x_tran) for each reach.

    Parameters:
        working_directory (str): Base path to the data directory.
        river_name (str): Name of the river.
        real_reach_lengths (pd.DataFrame): DataFrame containing real reach lengths.
        bulk_density (float): Bulk density of sediment in kg/m^3.

    Returns:
        pd.DataFrame: DataFrame with x_tran and n_stor values for each reach.
    """
    # Construct the path to the hydraulic geometry CSV
    hg_path = os.path.join(
        working_directory, "RiverMapping", "HydraulicGeometry", river_name, f"{river_name}_hydraulic_geometry.csv"
    )

    # Load hydraulic geometry data
    hydraulic_geometry = pd.read_csv(hg_path)
    hydraulic_geometry = hydraulic_geometry.rename(columns={"length_m": "GQBF_reach_length_m"})

    # Construct the path to the translation rate values CSV
    tr_path = os.path.join(
        working_directory, "RiverMapping", "Mobility", river_name, f"{river_name}_TR_values.csv"
    )

    # Load translation rate values
    tr_vals = pd.read_csv(tr_path)

    # Construct the path to the WBMsed data CSV
    wbmsed_path = os.path.join(
        working_directory, "WBMsed", "Extracted_Rivers", f"{river_name}_wbmsed.csv"
    )

    # Load WBMsed data
    wbmsed = pd.read_csv(wbmsed_path)

    # Merge all DataFrames on 'ds_order'
    merged_df = real_reach_lengths.merge(hydraulic_geometry, on="ds_order")
    merged_df = merged_df.merge(tr_vals, on="ds_order")
    merged_df = merged_df.merge(wbmsed, on="ds_order")

    # Convert sediment flux from kg/s to m^3/year using bulk density
    seconds_per_year = 365.25 * 24 * 60 * 60
    merged_df['sediment_flux_m3_yr'] = (
        (merged_df['mean_BedloadFlux_kg_s'] + merged_df['mean_SuspendedBedFlux_kg_s']) * seconds_per_year / bulk_density
    )

    # Compute x_tran in meters using sediment balance equation:
    # sediment_flux_m3_yr = x_tran * depth * width / TR
    # => x_tran = sediment_flux_m3_yr * TR / (depth * width)
    merged_df['x_tran_m'] = (
        merged_df['sediment_flux_m3_yr'] * merged_df['TR'] /
        (merged_df['depth_for_calcs_m'] * merged_df['median_width_m'])
    )

    # Compute n_stor as real_reach_length / x_tran
    merged_df['n_stor'] = merged_df['GQBF_reach_length_m'] / merged_df['x_tran_m']

    return merged_df


In [43]:
merged_df = calculate_xtran(working_directory, river_name, real_reach_lengths, 1600)

In [44]:
merged_df

,ds_order,real_reach_length_m,median_width_m,median_qbf_m3s,GQBF_reach_length_m,slope,BASED_depth_m,Repasch_depth_m,depth_for_calcs_m,TR,mean_BedloadFlux_kg_s,mean_Discharge_cms,mean_ParticleSize_m,mean_SedimentFlux_kg_s,mean_SuspendedBedFlux_kg_s,mean_WashloadFlux_kg_s,sediment_flux_m3_yr,x_tran_m,n_stor
0,20,64178.626636,139.7060,1730.568813,99514.21908,-0.000116,7.93,6.83,6.83,14.502269,4.344780,67.718285,0.002921,1337.27620,21.882465,1332.56670,5.172931e+05,7862.069282,12.657510
1,19,63825.781514,90.0000,1672.452543,109795.73520,-0.000068,10.48,6.83,6.83,17.287786,4.311036,66.369965,0.003491,1333.46230,22.368467,1311.73460,5.262132e+05,14799.188130,7.419038
2,18,63475.973818,54.8528,1547.825064,129767.13740,-0.000115,9.97,6.83,6.83,18.480821,4.255297,65.269400,0.000043,1327.49290,22.556349,1305.36910,5.288195e+05,26086.104101,4.974569
3,17,63302.532922,54.8528,1526.534022,139050.87480,-0.000154,9.90,6.83,6.83,7.973274,4.173318,63.869514,0.000009,1308.14870,22.784876,1285.59240,5.317099e+05,11315.974006,12.288016
4,16,62962.332716,139.7060,1802.095809,113461.66080,-0.000217,8.16,7.40,7.40,4.378868,7.700267,63.719204,0.001549,1299.36050,46.763718,1276.57570,1.074220e+06,4549.969884,24.936794
5,15,46977.787324,150.0000,1800.739923,99857.88578,-0.000169,8.04,7.40,7.40,4.078410,7.690269,63.703148,0.000027,1090.59690,46.756855,1128.39810,1.073888e+06,3945.724913,25.307868
6,14,46823.985123,104.1640,1682.509528,92277.58556,-0.000173,8.23,7.40,7.40,2.918763,6.672576,61.543380,0.000047,881.83325,41.406580,980.22046,9.482892e+05,3590.791954,25.698394
7,13,46389.296791,104.1640,1677.331533,89521.76255,-0.000170,8.26,7.40,7.40,5.586735,6.655877,59.383617,0.000047,873.44940,41.406580,832.04285,9.479599e+05,6870.662581,13.029568
8,12,46212.078380,104.1640,1654.524759,91801.25612,-0.000106,8.26,7.40,7.40,5.524988,6.635383,59.383617,0.000027,863.60626,41.408573,822.20496,9.475950e+05,6792.109908,13.515867
9,11,45389.961886,150.0000,1789.675492,83525.10536,-0.000137,8.04,7.40,7.40,5.568840,8.236425,59.383617,0.000038,855.72577,53.515553,814.38293,1.217965e+06,6110.497665,13.669117


In [45]:
def process_transit_lengths(
    working_directory: str,
    river_name: str,
    real_reach_lengths: pd.DataFrame,
    bulk_density: float
) -> None:
    """
    Wrapper function to calculate x_tran values and save them to CSV.

    Parameters:
        working_directory (str): Base path to the data directory.
        river_name (str): Name of the river.
        real_reach_lengths (pd.DataFrame): DataFrame containing real reach lengths.
        bulk_density (float): Bulk density of sediment in kg/m^3.
    """
    output_df = calculate_xtran(working_directory, river_name, real_reach_lengths, bulk_density)

    # Construct output path
    output_path = os.path.join(
        working_directory, "RiverMapping", "Mobility", river_name, f"{river_name}_transit_lengths.csv"
    )

    # Ensure directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Write to CSV
    output_df.to_csv(output_path, index=False)
    print(f"Transit data saved to: {output_path}")


In [46]:
process_transit_lengths(working_directory, river_name, real_reach_lengths, 1600)

Transit data saved to: D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\Bermejo_transit_lengths.csv
